# T-matrix computation for 3D acoustic scattering 

In [1]:
import bempp.api
import numpy as np
import scipy
from numba import objmode
import numba
import math
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
h = 0.3 # Size of the mesh
d = [1,0,0] # wave's travel direction
k = 5 # wavenumber

# far field points: unit sphere
unit_sphere = bempp.api.shapes.sphere(r = 1, origin=(0,0,0), h = h)
vert_sphere = unit_sphere.vertices
space_sphere = bempp.api.function_space(unit_sphere, 'P', 1)

# scatterer: cube
dom_cube = bempp.api.shapes.cube(length = 2/1.8, origin=(-1/1.8,-1/1.8,-1/1.8), h = h)
vert_cube = dom_cube.vertices
space_cube = bempp.api.function_space(dom_cube,'P',1)

In [ ]:
eta = k

identity = bempp.api.operators.boundary.sparse.identity(space_cube, space_cube,space_cube)
slp = bempp.api.operators.boundary.helmholtz.single_layer(space_cube,space_cube,space_cube,k)
adlp = bempp.api.operators.boundary.helmholtz.adjoint_double_layer(space_cube, space_cube,space_cube,k)

lhs = 0.5 * identity + adlp - 1j* eta *slp

In [ ]:
def normalized_spherical_harmonics(p, q, x):
    """Spherical Harmonic function of degree q"""
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/np.linalg.norm(x))
    if p >= 0:
        return ((-1)**p) * scipy.special.sph_harm(p,q,azimuth,polar)
    else:
        return scipy.special.sph_harm(-p,q,azimuth,polar)*np.exp(1j*2*p*azimuth)

def spherical_bessel_function(q,x):
    """Spherical Bessel function of degree q"""
    r = np.linalg.norm(x)
    return scipy.special.spherical_jn(q,k*r)


def spherical_hankel_function(q,x):
    """Spherical hankel function of degree q"""
    r = np.linalg.norm(x)
    return scipy.special.spherical_jn(q, k*r) + 1j * scipy.special.spherical_yn(q, k*r)

def regular_spherical_wavefunctions(p,q,x):
    """Regular Spherical Wavefunction"""
    return spherical_bessel_function(q,x)*normalized_spherical_harmonics(p,q,x)

def radiating_spherical_wavefunctions(p,q,x):
    """Radiating Spherical Wavefunction"""
    return spherical_hankel_function(q,x)*normalized_spherical_harmonics(p,q,x)

In [ ]:
def normalized_spherical_harmonics_dpolar(p, q, x):
    polar = np.arccos(x[2]/np.linalg.norm(x))
    azimuth = np.arctan2(x[1],x[0])
    y1 = normalized_spherical_harmonics(p, q, x)
    y3 = normalized_spherical_harmonics(-p, q, x)
    
    if abs(p) == q and p >= 0:
        return (p / np.tan(polar)) * y1 
    elif abs(p) != q and p >= 0:
        y2 = normalized_spherical_harmonics(p + 1, q, x)
        return (p / np.tan(polar)) * y1 + np.sqrt((q - p) * (q + p + 1)) * np.exp(-1j * azimuth) * y2
    elif abs(p) == q and p < 0:
        return (-p / np.tan(polar)) * y3 * np.exp(1j*p*azimuth*2) / (-1)**(-p)
    elif abs(p) != q and p < 0:
        y4 = normalized_spherical_harmonics(-p + 1, q, x)
        return ((-p / np.tan(polar)) * y3 + np.sqrt((q - (-p)) * (q + (-p) + 1)) * np.exp(-1j * azimuth) * y4) * (np.exp(1j*p*azimuth*2) / ((-1)**(-p)))

def normalized_spherical_harmonics_dazimuth(p, q, x):
    return 1j * p * normalized_spherical_harmonics(p, q, x)

def spherical_bessel_function_dr(q,x):
    r = np.linalg.norm(x)
    return k*(-spherical_bessel_function(q+1,x) + q/(k*r)*spherical_bessel_function(q,x))

In [ ]:
deg = 10
slp_far_field_coeff = []

for q in range(deg+1):
    for p in range(-q,q+1):
        @bempp.api.complex_callable
        def rhs_data(x, n, domain_index, result):
            with objmode():
                r = np.linalg.norm(x)
                azimuth = np.arctan2(x[1],x[0])
                polar = np.arccos(x[2]/r)

                r_ = spherical_bessel_function_dr(q,x)*normalized_spherical_harmonics(p,q,x)
                polar_ = (1/r)*normalized_spherical_harmonics_dpolar(p,q,x)*spherical_bessel_function(q,x)
                azimuth_ = (1/(r*np.sin(polar)))*normalized_spherical_harmonics_dazimuth(p,q,x)*spherical_bessel_function(q,x)

                x0 = r_*np.sin(polar_)*np.cos(azimuth_)
                x1 = r_*np.sin(polar_)*np.sin(azimuth_)
                x2 = r_*np.cos(polar_)

                sph_to_cart = [x0, x1, x2]                      
                normal = [n[0],n[1],n[2]]

                result[0] = np.inner(sph_to_cart, normal) - 1j * eta * regular_spherical_wavefunctions(p,q,x)

        rhs_fun = bempp.api.GridFunction(space_cube, fun = rhs_data)
        neumann_total_field, info = bempp.api.linalg.gmres(lhs, rhs_fun, tol=1E-5)
        slp_far_field = bempp.api.operators.far_field.helmholtz.single_layer(space_cube,vert_sphere,k)
        slp_far_field_coeff.append(- slp_far_field.evaluate(neumann_total_field))

In [ ]:
sph_harm_coeff = []

for q in range(deg+1):
    for p in range(-q,q+1):
        @bempp.api.complex_callable
        def sph_harm(x,n,domain_index,result):
            with objmode():
                result[0] = normalized_spherical_harmonics(p,q,x)
        sph_harm_grid = bempp.api.GridFunction(space_sphere,fun = sph_harm)
        sph_harm_coeff.append(sph_harm_grid.coefficients)

In [ ]:
mass_mat = bempp.api.operators.boundary.sparse.identity(space_sphere,space_sphere,space_sphere).weak_form().A

T_matrix = np.zeros(((deg+1)**2,(deg+1)**2),dtype = complex)
for i in range((deg+1)**2):
    for j in range((deg+1)**2):
        T_matrix[i,j] = slp_far_field_coeff[j] @ mass_mat @ np.conj(sph_harm_coeff[i])

### Test the T-matrix 

In [ ]:
def coeff_sph_expansion(p,q):
    """Analytical formula of the coefficients of the spherical expansions of plane waves"""
    coeff = 4*np.pi*(1j)**q*normalized_spherical_harmonics(-p,q,d)/(-1)**(p + abs(p))
    return coeff

In [ ]:
coeff_inc = []
for q in range(deg+1):
    for p in range(-q, q+1):
        coeff_inc.append(coeff_sph_expansion(p,q))

In [ ]:
coeff_sca = T_matrix @ coeff_inc

In [ ]:
def sca_far_field(x):
    fun = 0
    i = 0
    for q in range(deg+1):
        for p in range(-q,q+1):
            fun += coeff_sca[i]*(1/k)*(-1j)**(q+1) * normalized_spherical_harmonics(p, q, x)
            i +=1
    return fun

This is the grid function of the far field pattern of the scattering field on the sphere.

In [ ]:
@bempp.api.complex_callable
def sca_far_field_func(x, n, domain_index, result):
    with objmode():
        result[0] = sca_far_field(x)
    
sca_far_field_grid_func = bempp.api.GridFunction(space_sphere, fun=sca_far_field_func)

Compute the exact scattered field on the cube and find the far field of it on the sphere.

In [ ]:
@bempp.api.complex_callable
def combined_data(x, n, domain_index, result):
    result[0] = 1j * k * np.exp(1j * k * x[0]) * n[0] - 1j * eta * np.exp(1j * k * x[0])

rhs_fun_exact = bempp.api.GridFunction(space_cube, fun=combined_data)
neumann_total_field_exact, info_exact = bempp.api.linalg.gmres(lhs,rhs_fun_exact,tol=1E-5)
slp_far_field_exact = bempp.api.operators.far_field.helmholtz.single_layer(space_cube,vert_sphere,k)
slp_far_field_coeff_exact = - slp_far_field_exact.evaluate(neumann_total_field_exact)

Compute the l2 norm of the difference $u_{exact}^{s} - u_{appro}^{s}$ in the far field.  
$$||u_{exact}^{s} - u_{appro}^{s}||_{l_{2}(sphere)} = <u_{exact}^{s} - u_{appro}^{s}, u_{exact}^{s} - u_{appro}^{s}>^{\frac{1}{2}} = <\delta u, \delta u>^{\frac{1}{2}} = \int \delta u\delta u ^{*}dx$$

In [ ]:
error_coeff_test1 = slp_far_field_coeff_exact - sca_far_field_grid_func.coefficients
error_coeff_test2 = np.conj(slp_far_field_coeff_exact) - np.conj(sca_far_field_grid_func.coefficients)
error_test = np.sqrt(error_coeff_test1 @ mass_mat @ error_coeff_test2.T)

In [ ]:
error_test